# Step 1. 데이터 다운로드

data 폴더에 aiffel에서 받은 데이터 파일을 저장해둠. 

# Step 2. 데이터 읽어오기


In [2]:
import glob
import os

txt_file_path = './data/*' # 데이터 위치

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 4: illegal multibyte sequence

"UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 4: illegal multibyte sequence"

코덱 읽기 오류가 발생했는데 이는 아래와 같이 코드를 수정하면 해결된다. 

In [2]:
import glob
import os

txt_file_path = './data/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기.
for txt_file in txt_list:
    with open(txt_file, 'rt', encoding='UTF8') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", 'Had some bad love', "So fast love is all that I've got on my mind Ooh, ooh", 'Ooh, ooh Looking for some affirmation', 'Made my way into the sun', 'My friends got their ladies', "And they're all having babies", "I just wanna have some fun I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", "I've had some bad love", "So fast love is all that I've got on my mind Ooh, ooh", 'Baby, baby'

정상적으로 데이터를 읽어온 것을 확인할 수 있다. 

# Step 3. 데이터 정제


In [3]:
# 어떤 문장이 들어왔는지 확인
print(raw_corpus[:9])

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes"]


여기서 볼 수 있는 건 축약어로 don't라던가 Let's 등의 표현이 자주 보인다는 점이다. 따라서 기존 코드에 ' <- 이 작은따옴표는 정제할 때 추가로 제외 시켜야한다.  

In [4]:
!pip install tensorflow

In [5]:
import os, re 
import numpy as np
import tensorflow as tf

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿,'가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있다.
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?'.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    
    # bruce-springsteen.txt에서 [Verse] 와 [Chorus]라는 구분자가 보임. 이는 문장이 아니므로 삭제함.
    if "verse" in sentence: # 1번에서 이미 sentence는 소문자가 되었기 때문에 소문자로 필터링
        sentence = sentence.replace("verse", "")
    if "chorus" in sentence:# 위와 동일
        sentence = sentence.replace("chorus", "")
        
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample's        sentence."))

<start> this is sample's sentence . <end>


작은따옴표도 제외하고 잘 필터링 되는 것을 확인할 수 있다.

In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue # 아무것도 없는 문장
    if sentence[-1] == ":": continue # 대화체 구분자
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 30개만 확인해보죠
corpus[:30]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 "<start> baby , can't you read the signs ? i won't bore you with the details , baby <end>",
 "<start> i don't even wanna waste your time <end>",
 "<start> let's just say that maybe <end>",
 '<start> you could help me ease my mind <end>',
 "<start> i ain't mr . right but if you're looking for fast love <end>",
 "<start> if that's love in your eyes <end>",
 "<start> it's more than enough <end>",
 '<start> had some bad love <end>',
 "<start> so fast love is all that i've got on my mind ooh , ooh <end>",
 '<start> ooh , ooh looking for some affirmation <end>',
 '<start> made my way into the sun <end>',
 '<start> my friends got their ladies <end>',
 "<start> and they're all having babies <end>",
 "<start> i just wanna have some fun i won't bore you with the details , baby <end>",
 "<start> i don't even wanna waste your time <end>",
 "<start> let's just 

In [7]:
len(corpus)

175749

175749 개의 정제된 문장이 준비되었다. 

우리는 여기에 너무 긴 문장은 제외할 것이다. 
- 여기서 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외할 것이다.
- 단어장 크기는 12000단어이다.

In [8]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    
    for num in tensor:
        if len(num) >= 17: # <start>, <end> 포함하여 17단어 이상(빼면 15단어 이상) 
            tensor = np.delete(tensor, num) # 너무 긴 문장은 삭제
            break;
           
    
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=16) #위에서 17단어 이상 걸렀으니 최대 16 
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 310  26 ...   0   0   0]
 [  2 226  12 ...   0   0   0]
 [  2  48 121 ...   0   0   0]
 ...
 [  2  25  71 ...   0   0   0]
 [  2  38  23 ...   0   0   0]
 [  2  25  71 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x000002374CA43CD0>


In [9]:
print(len(tensor))

175734


길이가 15단어 이상인 문장은 삭제되어 175749개 -> 175734개의 문장으로 정제 되었다.

In [10]:
# 어떻게 토큰이 구성되었는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 35: break # 35번까지만 확인

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : to
10 : a
11 : me
12 : my
13 : it
14 : in
15 : that
16 : on
17 : of
18 : .
19 : your
20 : i'm
21 : like
22 : all
23 : is
24 : be
25 : we
26 : for
27 : up
28 : so
29 : with
30 : know
31 : love
32 : just
33 : but
34 : don't
35 : no


In [11]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  310   26   94 4918    3    0    0    0    0    0    0    0    0
    0]
[ 310   26   94 4918    3    0    0    0    0    0    0    0    0    0
    0]


토큰 하나를 잘라내서 길이가 14이다.

In [12]:
#데이터 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

# Step 4. 평가 데이터셋 분리

 단어장의 크기는 12,000 이상 으로 설정! 총 데이터의 20% 를 평가 데이터셋으로 사용

In [12]:
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=664ac2c98cfbc71e1429ac3f6a1a10a6cdbf42bbd75ed9cfe32b258b0bc5fe64
  Stored in directory: c:\users\s_kmh0207\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [13]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          train_size = 0.8) # 20%를 평가 데이터셋으로 사용하므로 0.8

In [14]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140587, 15)
Target Train: (140587, 15)


14만여개의 데이터를 기준으로 학습을 진행한다. enc_train과 dec_train이 x_train, y_train에 대응한다.

# Step 5. 인공지능 만들기

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니
model(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[ 5.30079524e-05,  1.72507149e-04, -2.67790801e-05, ...,
          1.88825972e-04, -1.82728603e-04, -1.09168090e-04],
        [-1.11650043e-05,  5.09861275e-05, -6.87651700e-05, ...,
          4.34595771e-04, -2.15719090e-04, -4.30559303e-04],
        [ 3.40703642e-04, -2.08054735e-05,  1.00562174e-04, ...,
          3.75593518e-04, -2.89088202e-04, -5.42103371e-04],
        ...,
        [-7.88067351e-04,  1.57482736e-03, -1.70152925e-04, ...,
         -1.00865657e-03,  5.41033631e-04, -1.28268322e-03],
        [-1.10363984e-03,  1.89268892e-03, -3.24623979e-04, ...,
         -9.73247923e-04,  8.74658290e-04, -1.68963871e-03],
        [-1.38430740e-03,  2.15180730e-03, -4.67916369e-04, ...,
         -9.43981868e-04,  1.16513437e-03, -2.04160390e-03]],

       [[ 5.30079524e-05,  1.72507149e-04, -2.67790801e-05, ...,
          1.88825972e-04, -1.82728603e-04, -1.09168090e-04],
        [ 2.89653253e-04,  1.77729380e-04, -2

In [17]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=(enc_val, dec_val), epochs=10) # 노트북으로 짆행시 시간이 오래 결려 PC 환경에서 진행하도록함. 

Epoch 1/10
686/686 [==============================] - 123s 179ms/step - loss: 3.4166 - val_loss: 3.0842
Epoch 2/10
686/686 [==============================] - 119s 174ms/step - loss: 2.9939 - val_loss: 2.8688
Epoch 3/10
686/686 [==============================] - 119s 174ms/step - loss: 2.8117 - val_loss: 2.7020
Epoch 4/10
686/686 [==============================] - 119s 174ms/step - loss: 2.6675 - val_loss: 2.5655
Epoch 5/10
686/686 [==============================] - 119s 174ms/step - loss: 2.5442 - val_loss: 2.4506
Epoch 6/10
686/686 [==============================] - 119s 174ms/step - loss: 2.4327 - val_loss: 2.3373
Epoch 7/10
686/686 [==============================] - 119s 174ms/step - loss: 2.3305 - val_loss: 2.2324
Epoch 8/10
686/686 [==============================] - 119s 174ms/step - loss: 2.2340 - val_loss: 2.1412
Epoch 9/10
686/686 [==============================] - 119s 174ms/step - loss: 2.1439 - val_loss: 2.0506
Epoch 10/10
686/686 [==============================] - 119s 174m

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , liberian girl <end> '

그럴 듯한 문장이 생성되었다.

추가로 나눈 학습데이터로 또 진행해보자.

In [32]:
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

#나눈 덱이터방식으로도 진행해봄.
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=10, batch_size=100) 


Epoch 1/10
1406/1406 [==============================] - 84s 60ms/step - loss: 3.5017 - val_loss: 3.1823
Epoch 2/10
1406/1406 [==============================] - 84s 60ms/step - loss: 3.0419 - val_loss: 2.9611
Epoch 3/10
1406/1406 [==============================] - 85s 60ms/step - loss: 2.8085 - val_loss: 2.8228
Epoch 4/10
1406/1406 [==============================] - 85s 60ms/step - loss: 2.6084 - val_loss: 2.7269
Epoch 5/10
1406/1406 [==============================] - 85s 60ms/step - loss: 2.4257 - val_loss: 2.6565
Epoch 6/10
1406/1406 [==============================] - 85s 60ms/step - loss: 2.2574 - val_loss: 2.6055
Epoch 7/10
1406/1406 [==============================] - 85s 60ms/step - loss: 2.1031 - val_loss: 2.5758
Epoch 8/10
1406/1406 [==============================] - 85s 60ms/step - loss: 1.9601 - val_loss: 2.5497
Epoch 9/10
1406/1406 [==============================] - 85s 61ms/step - loss: 1.8286 - val_loss: 2.5335
Epoch 10/10
1406/1406 [==============================] - 85s 60m

In [33]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , i love you , i love you <end> '

In [35]:
#나눈 덱이터방식으로도 진행해봄.
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=10, batch_size=300) 

Epoch 1/10
469/469 [==============================] - 67s 143ms/step - loss: 1.5712 - val_loss: 2.5175
Epoch 2/10
469/469 [==============================] - 68s 145ms/step - loss: 1.4519 - val_loss: 2.5217
Epoch 3/10
469/469 [==============================] - 68s 145ms/step - loss: 1.3624 - val_loss: 2.5285
Epoch 4/10
469/469 [==============================] - 68s 146ms/step - loss: 1.2898 - val_loss: 2.5367
Epoch 5/10
469/469 [==============================] - 68s 145ms/step - loss: 1.2262 - val_loss: 2.5494
Epoch 6/10
469/469 [==============================] - 69s 146ms/step - loss: 1.1704 - val_loss: 2.5629
Epoch 7/10
469/469 [==============================] - 69s 148ms/step - loss: 1.1215 - val_loss: 2.5825
Epoch 8/10
469/469 [==============================] - 69s 147ms/step - loss: 1.0789 - val_loss: 2.6001
Epoch 9/10
469/469 [==============================] - 69s 147ms/step - loss: 1.0420 - val_loss: 2.6218
Epoch 10/10
469/469 [==============================] - 69s 146ms/step - l

In [36]:
generate_text(model, tokenizer, init_sentence="<start> i love")

"<start> i love you , i'm not gonna crack <end> "

In [37]:
#batch_size == 200
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=10, batch_size=200) 

Epoch 1/10
703/703 [==============================] - 76s 108ms/step - loss: 0.9875 - val_loss: 2.6630
Epoch 2/10
703/703 [==============================] - 76s 108ms/step - loss: 0.9714 - val_loss: 2.6920
Epoch 3/10
703/703 [==============================] - 76s 108ms/step - loss: 0.9539 - val_loss: 2.7166
Epoch 4/10
703/703 [==============================] - 76s 108ms/step - loss: 0.9378 - val_loss: 2.7369
Epoch 5/10
703/703 [==============================] - 76s 108ms/step - loss: 0.9211 - val_loss: 2.7613
Epoch 6/10
703/703 [==============================] - 76s 109ms/step - loss: 0.9077 - val_loss: 2.7849
Epoch 7/10
703/703 [==============================] - 76s 109ms/step - loss: 0.8969 - val_loss: 2.8046
Epoch 8/10
703/703 [==============================] - 76s 109ms/step - loss: 0.8874 - val_loss: 2.8249
Epoch 9/10
703/703 [==============================] - 76s 108ms/step - loss: 0.8791 - val_loss: 2.8439
Epoch 10/10
703/703 [==============================] - 75s 107ms/step - l

In [38]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love the way you lie <end> '

아무래도 embedding_size와 hidden_size를 좀 변동해야할 것같다. 

떨어지던 val_loss가 늘어나는 것으로봐선 현재 과적합 문제가 발생하고 있는것으로 보인다. 

배치 사이즈는 가장 효과가 괜찮은 300으로 잡도록한다. 

In [39]:
embedding_size = 128
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

In [40]:
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=10, batch_size=300) 

Epoch 1/10
469/469 [==============================] - 183s 391ms/step - loss: 3.6838 - val_loss: 3.2660
Epoch 2/10
469/469 [==============================] - 185s 394ms/step - loss: 3.1048 - val_loss: 2.9887
Epoch 3/10
469/469 [==============================] - 185s 395ms/step - loss: 2.8027 - val_loss: 2.7944
Epoch 4/10
469/469 [==============================] - 185s 395ms/step - loss: 2.5075 - val_loss: 2.6352
Epoch 5/10
469/469 [==============================] - 187s 399ms/step - loss: 2.2173 - val_loss: 2.5181
Epoch 6/10
469/469 [==============================] - 185s 395ms/step - loss: 1.9439 - val_loss: 2.4358
Epoch 7/10
469/469 [==============================] - 185s 395ms/step - loss: 1.6989 - val_loss: 2.3863
Epoch 8/10
469/469 [==============================] - 185s 395ms/step - loss: 1.4908 - val_loss: 2.3564
Epoch 9/10
469/469 [==============================] - 186s 397ms/step - loss: 1.3209 - val_loss: 2.3499
Epoch 10/10
469/469 [==============================] - 187s 399m

In [41]:
generate_text(model, tokenizer, init_sentence="<start> i love")

"<start> i love you , i'm not gonna crack <end> "

In [1]:
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

NameError: name 'TextGenerator' is not defined

In [ ]:
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=10, batch_size=300) 

In [47]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you <end> '

# 마무리



기존 실습 방식대로 진행했을 때, 
텍스트 생성모델의 10 epoch 쯤 loss는 2.0967으로 2.2보다 낮다.
또한 그럴듯한 문장이 잘 생성된 것을 확인했다. 

loss 값의 경우 epoch를 50으로 내리면 계속 내려갈 것으로 보인다. 


하지만 이건 val_loss를 본 것이 아니므로 추가로 학습을 진행했는데, 
val_loss가 점점 늘어나는 것으로 보아 배치 크기만으로 낮추는 것은 어려운 것으로 보았다.
(첫 학습 배치 사이즈 200은 2.2보다 아래였으나, 아마 이전에 dataset으로 학습을 미리한 것에 덮어써서 그런 결과가 나온 것 같았다.)

몇몇 논문과 학습시간을 고려하여 배치 사이즈는 300으로 고정하였고, 
embedding_size와 hidden_size를 조절하여 epoch 10 이내에 val_loss가 2.2보다 아래가 되도록 조정해보았다.

우선 hidden_size가 커지면 커질 수록 한 Epoch마다 걸리는 학습시간이 몇배 늘어나게 된다. 
따라서 hidden_size는 크기를 유지하거나 낮추는 방향으로 가고 embedding_size를 조절하는 것이 맞다고 생각했다.

근데 embedding_size는 ouput_size와 동의어로 보인다. 
총 15단어 미만의 문장이 나오기 때문에 embedding_size는 앞 뒤 <start>와 <end>를 합쳐 16이 되는 것이 맞을 것이다. 

다양한 수치를 대입해보아 val_loss를 2.2보다 낮게 만들려고 시도했다.
    embedding_size = 256, hidden_size = 2048로 2.3 내외로 만든게 가장 작다. 


이번 프로젝트에서는 문장을 생성하는 모델을 어떻게 생성하는지 다시 확인해볼 수 있었다.

그런데 평가 데이터셋을 분리하였는데, 정작 model에 fit한 것은 이전에 만든 데이터셋을 그대로 활용하였다. 
결과에선 큰 문제가 없을 거 같은데 추가로 학습을 짆애해 보아야겠다. 

중간 나온 오류들에 대한 정리는 아래 링크에 작성해두었다. 
https://citrine-cashew-534.notion.site/1-13-1b1b5b032f1049f58b993f5a6b4a2e7d